In [1]:
import pandas as pd
import string
import re
import pickle

from nltk.corpus import stopwords

## Define Functions to clean text ##

1. Remove strange characters
2. Remove punctuations
3. Change all characters to lower case
4. Remove stopwords
5. Remove numerical characters

In [2]:
def mod_code(text):
    text = re.sub(r'[^\x00-\x7f]',r'', text)
    return text

def mod_char(text):
    text = " ".join(word.replace('[^\w\s]',' ') for word in text.split())
    return text

def mod_punctuations(text):
    for punctuation in string.punctuation:
        text = " ".join(word.replace(punctuation, '') for word in text.split())
    return text

def mod_lower(text):
    text = text.lower()
    return text

stop = stopwords.words('english')
def mod_stop(text):
    text = ' '.join(word for word in text.split() if word not in stop)
    return text

def rem_num(text):
    text = ' '.join(word for word in text.split() if not word.isdigit())
    return text

def clean_df(df,col):
    df[col] = df[col].apply(rem_num)
    df[col] = df[col].apply(mod_char)
    df[col] = df[col].apply(mod_punctuations)
    df[col] = df[col].apply(mod_lower)
    df[col] = df[col].apply(mod_stop)
    df[col] = df[col].apply(mod_code)
    
def clean_txt(text):
    text = mod_char(text)
    text = mod_punctuations(text)
    text = mod_lower(text)
    text = mod_code(text)
    text = mod_stop(text)
    
    return text

## Load Text Data with Labels ##

ISEAR (International Survey on Emotion Antecedents and Reactions)

A compilation of 7,666 sentences provided by 1,096 culturally divergent participants who were questioned about experiences and reactions that related to the emotions of anger, disgust, fear, joy, sadness, and guilt.

Justification: 
The data will be suitable to train the model based on straightforward expressions of emotions.

In [10]:
isear = pd.read_csv('isear_org.txt',names=['labels','texts','non'], )
isear.head()

,labels,texts,non
0,joy,On days when I feel close to my partner and ot...,NaN
1,fear,Every time I imagine that someone I love or I ...,NaN
2,anger,When I had been obviously unjustly treated and...,NaN
3,sadness,When I think about the short time that we live...,NaN
4,disgust,At a gathering I found myself involuntarily si...,NaN


In [11]:
isear.drop(['non'], axis=1, inplace=True)
isear['labels'] = isear['labels'].apply(lambda x: 'guilt' if x == 'guit' else x)

In [12]:
isear.to_csv("isear.csv")

In [13]:
clean_df(isear,'texts')
isear.head()

/Users/bokshenggan/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:20: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


,labels,texts
0,joy,days feel close partner friends feel peace als...
1,fear,every time imagine someone love could contact ...
2,anger,obviously unjustly treated possibility elucida...
3,sadness,think short time live relate periods life thin...
4,disgust,gathering found involuntarily sitting next two...


## Explore classes and proportions

Classess are balanaced

In [14]:
pd.value_counts(isear['labels'])

joy        1092
sadness    1082
anger      1079
fear       1076
shame      1071
disgust    1066
guilt      1050
Name: labels, dtype: int64

## Save in pickle

In [15]:
isear.to_pickle("../1_Baseline/isear.pkl")